In [16]:
import requests
import json
import datetime
import pandas as pd
import pytz

class FastGenie:
    
    def __init__(self, location, start_date):
        self.location = location
        self.start_date = start_date
        self.description = "This is very simple tool to generate a time table for nerdy Baha'is during the fast."
        
    def geo_api(self):
        GEO_API = 'https://api.opencagedata.com/geocode/v1/json?'
        api_out = requests.get(GEO_API, params={'q': self.location,
                              'key': '9885520318ca426793c0d92a965d5b1b',
                              'language': 'en',
                              'pretty': 1}).json()['results']
        
        lat,lng = api_out[0]['geometry']['lat'],api_out[0]['geometry']['lng']
        
        return lat,lng
    
    def get_sun_schedule(self):
        lat,lng = self.geo_api()
        SUNRISE_API = 'https://api.sunrise-sunset.org/json?'
        series_date = pd.date_range(self.start_date, freq='D', periods=21)
        datelist = series_date.astype(str)
        
        output = list()
        
        # Easier to loop to collect the date, too
        for date in datelist:
            api_out = requests.get(SUNRISE_API, params={'lat': lat,
                                                        'lng': lng,
                                                        'date': date,
                                                        'formatted': 0}).json()['results']
            api_out['date'] = date
            api_out['day_of_week'] = datetime.datetime.strptime(date, '%Y-%m-%d').strftime('%A')
            output.append(api_out)
        return output

    def get_time_table(self):
        good_data = self.get_sun_schedule()
        df = pd.DataFrame(good_data)
        df = df[['day_of_week','date','sunrise','sunset']]
        
        # Set timezone
        tz = pytz.timezone('US/Pacific')
        df['sunrise'] = pd.to_datetime(df['sunrise']).apply(lambda x: tz.fromutc(x))
        df['sunset'] = pd.to_datetime(df['sunset']).apply(lambda x: tz.fromutc(x))
        
        # Beautify
        df['sunrise'] = df['sunrise'].apply(lambda x: x.strftime('%I:%M:%S %p'))
        df['sunset'] = df['sunset'].apply(lambda x: x.strftime('%I:%M:%S %p'))
        
        return df

## Enter your address and start date, and run this baby!

In [17]:
YOUR_ADDRESS = '1451 Eden Ave San Jose CA 95117'
FIRST_DAY_OF_FAST = '2019-03-01'
FastPrinter = FastGenie(YOUR_ADDRESS,FIRST_DAY_OF_FAST)
FastPrinter.get_time_table()

,day_of_week,date,sunrise,sunset
0,Friday,2019-03-01,06:37:59 AM,06:02:04 PM
1,Saturday,2019-03-02,06:36:36 AM,06:03:02 PM
2,Sunday,2019-03-03,06:35:12 AM,06:04:01 PM
3,Monday,2019-03-04,06:33:48 AM,06:04:59 PM
4,Tuesday,2019-03-05,06:32:23 AM,06:05:57 PM
5,Wednesday,2019-03-06,06:30:57 AM,06:06:54 PM
6,Thursday,2019-03-07,06:29:31 AM,06:07:51 PM
7,Friday,2019-03-08,06:28:04 AM,06:08:48 PM
8,Saturday,2019-03-09,06:26:37 AM,06:09:45 PM
9,Sunday,2019-03-10,07:25:10 AM,07:10:42 PM
